In [ ]:
from collections import namedtuple
import csv
import json
import logging
import tarfile
from time import time
from typing import List

from minio import Minio
from minio.error import S3Error
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

pd.options.mode.chained_assignment = None
bucket_name = 'lake'

In [ ]:
def tar_to_df(bucket_name: str, object_name: str) -> pd.DataFrame:
    '''
    This function will take a tarfile reference in MinIO and do the following:
        - unzip the tarfile
        - turn the data into a single DataFrame object
    '''
    logger = logging.getLogger('gsod_logger')
    logger.setLevel(logging.INFO)

    # Temp file to use for processing the tar files.
    temp_file_name = 'temp.tar.gz'
    # Load the credentials and connection information.
    with open('credentials.json') as f:
        credentials = json.load(f)

    # Get data of an object.
    try:
        # Create client with access and secret key
        client = Minio(credentials['url'],  # host.docker.internal
                    credentials['accessKey'],  
                    credentials['secretKey'], 
                    secure=False)
        
        object_info = client.fget_object(bucket_name, object_name, temp_file_name)

        Row = namedtuple('Row', ('station', 'date', 'latitude', 'longitude', 'elevation', 'name', 'temp', 'temp_attributes', 'dewp',
                                'dewp_attributes', 'slp', 'SLP_attributes', 'stp', 'stp_attributes', 'visib', 'visib_attributes',
                                'wdsp', 'wdsp_attributes', 'mxspd', 'gust', 'max', 'max_attributes', 'min', 'min_attributes', 'prcp',
                                'prcp_attributes', 'sndp', 'frshtt'))
        
        # Columns of interest and their data types.
        dtypes={
            'station': 'string',
            'date': 'datetime64[ns]',
            'latitude': 'float64',
            'longitude': 'float64',
            'name': 'string',
            'temp': 'float64'
        }

        tar = tarfile.open(temp_file_name, 'r:gz')
        all_rows = []
        for member in tar.getmembers():
            member_handle = tar.extractfile(member)
            byte_data = member_handle.read()
            decoded_string = byte_data.decode()
            lines = decoded_string.splitlines()
            reader = csv.reader(lines, delimiter=',')

            # Get all the rows in the member. Skip the header.
            _ = next(reader)
            file_rows = [Row(*l) for l in reader]
            all_rows += file_rows

        df = pd.DataFrame.from_records(all_rows, columns=Row._fields)
        
        df = df[list(dtypes.keys())]
        for c in df.columns:
            if dtypes[c] == 'float64': df[c] = pd.to_numeric(df[c], errors='coerce')
        df = df.astype(dtype=dtypes)

    except S3Error as s3_err:
        logger.error(f'S3 Error occurred: {s3_err}.')
        raise s3_err
    except Exception as err:
        logger.error(f'Error occurred: {err}.')
        raise err

    return df

In [ ]:
def get_object_list(bucket_name: str, prefix: str) -> List[str]:
    '''
    Gets a list of objects from a bucket.
    '''
    logger = logging.getLogger('gsod_logger')
    logger.setLevel(logging.INFO)

    # Load the credentials and connection information.
    with open('credentials.json') as f:
        credentials = json.load(f)

    # Get data of an object.
    try:
        # Create client with access and secret key
        client = Minio(credentials['url'],  # host.docker.internal
                    credentials['accessKey'],  
                    credentials['secretKey'], 
                    secure=False)

        object_list = []
        objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
        for obj in objects:
            object_list.append(obj.object_name)
    except S3Error as s3_err:
        logger.error(f'S3 Error occurred: {s3_err}.')
        raise s3_err
    except Exception as err:
        logger.error(f'Error occurred: {err}.')
        raise err

    return object_list


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Jupyter').getOrCreate()

objects = get_object_list(bucket_name, 'noaa/gsod')

for obj in reversed(objects):
    print(obj)
    df = tar_to_df(bucket_name, obj)
    table = pa.Table.from_pandas(df)
    pq.write_table(table, 'temp.parquet')
    
    df = spark.read.parquet('temp.parquet')
    df.write.mode('append').saveAsTable('noaa.gsod')


In [ ]:
obj = 'noaa/gsod/2011.tar.gz'
df = tar_to_df(bucket_name, obj)
table = pa.Table.from_pandas(df)
pq.write_table(table, 'temp.parquet')
    
df = spark.read.parquet('temp.parquet')
df.write.mode('append').saveAsTable('noaa.gsod')
print(obj)